Service facility with two services in series, each with a FIFO queue. A customer leaving server 1 proceeds immediately to server 2 (or its queue), with no delay.

* Need two queues in memory.
* There will be four total events:
    * Arrival to the first server
    * Arrival to the second server
    * Departure from facility
    * Termination of simulation

1. Start with both Q1 and Q2 empty. When the first customer arrives, i.e., when an A1 event occurs, the customer immediately begins service, so the departure from S1 --- i.e., the A2 event --- is scheduled.
2. When the A2 event occurs:
  - Any customer waiting in Q1 will start service in S1; their A2 event is scheduled.
  - The customer of the A2 event will either start service in S2 or enter Q2.
    - If Q2 empty, this customer starts service in S2 and its Departure event is scheduled.
    - If Q2 nonempty, this customer enters Q2.
3. When the D event occurs, ...

As in the book's example, the Event List is stored in `TIME_NEXT_EVENT`

In [25]:
import numpy as np
from enum import Enum

MEAN_INTERARRIVAL = 1.0
MEAN_SERVICE_1 = 0.7
MEAN_SERVICE_2 = 0.9

NUM_EVENTS = 4
SIM_TIME = 0.0

SERVER_STATUS = []
TIME_ARRIVAL_1 = [] # queue 1
TIME_ARRIVAL_2 = [] # queue 2
TIME_NEXT_EVENT = [] # event list
TIME_LAST_EVENT = 0

NUM_IN_Q_1 = 0
NUM_IN_Q_2 = 0
TOTAL_OF_DELAYS_1 = 0.0
TOTAL_OF_DELAYS_2 = 0.0
NUM_CUSTS_DELAYED_1 = 0
NUM_CUSTS_DELAYED_2 = 0
AREA_NUM_IN_Q_1 = 0.0
AREA_NUM_IN_Q_2 = 0.0
AREA_SERVER_STATUS_1 = 0.0
AREA_SERVER_STATUS_2 = 0.0


def expon(mean): 
    return np.random.exponential(scale=mean)

class ServiceStatus(Enum):
    IDLE = 0
    BUSY = 1

def initialize():
    """Initialize simulation variables."""
    # init sim clock
    global SIM_TIME
    SIM_TIME = 0.0

    # init state vars
    global SERVER_STATUS
    SERVER_STATUS = [ServiceStatus.IDLE, ServiceStatus.IDLE]
    NUM_IN_Q_1 = 0
    NUM_IN_Q_2 = 0
    TIME_LAST_EVENT = 0.0

    AREA_NUM_IN_Q_1 = 0.0
    AREA_NUM_IN_Q_2 = 0.0
    AREA_SERVER_STATUS_1 = 0.0
    AREA_SERVER_STATUS_2 = 0.0

    # init event list
    global TIME_NEXT_EVENT, MEAN_INTERARRIVAL
    TIME_NEXT_EVENT = [
        SIM_TIME + expon(MEAN_INTERARRIVAL), # initial S1 arrival event
        np.inf, # initial S2 arrival event
        np.inf, # departure event
        1_000, #  termination time
    ]


def timing():
    """Compare the timings of the next of each possible event type to find
    the next event to occur. Set NEXT_EVENT_TYPE accordingly, and advance the
    simulation clock to the time of occurrence of this event.
    """
    global NEXT_EVENT_TYPE, NUM_EVENTS, TIME_NEXT_EVENT, SIM_TIME

    min_time_next_event = np.inf

    for i in range(NUM_EVENTS):
        if TIME_NEXT_EVENT[i] < min_time_next_event:
            min_time_next_event = TIME_NEXT_EVENT[i]
            NEXT_EVENT_TYPE = i
    SIM_TIME = TIME_NEXT_EVENT[NEXT_EVENT_TYPE]


def arrive1():
    """Handle the arrival of a customer to Server 1.
    If the server is busy, the customer enters Queue 1.
    If the server is idle, the customer immediately enters Service and
        schedules its departure/arrival to Queue 2.
    """
    delay = 0

    # schedule the next arrive1 event
    global TIME_NEXT_EVENT, SIM_TIME, MEAN_INTERARRIVAL
    TIME_NEXT_EVENT[0] = SIM_TIME + expon(MEAN_INTERARRIVAL)

    global SERVER_STATUS
    if SERVER_STATUS[0] is ServiceStatus.BUSY:
        # if S1 is busy, go to Queue 1
        global NUM_IN_Q_1
        NUM_IN_Q_1 += 1

        # check for queue overflow
        # ...

        # store arrival time of this customer at end of queue
        global TIME_ARRIVAL_1
        TIME_ARRIVAL_1 += [SIM_TIME]

    else:
        # if S1 is idle, go directly to service
        delay = 0.0
        global TOTAL_OF_DELAYS_1, NUM_CUSTS_DELAYED_1
        TOTAL_OF_DELAYS_1 += delay
        NUM_CUSTS_DELAYED_1 += 1

        # make Server 1 busy
        SERVER_STATUS[0] = ServiceStatus.BUSY

        # schedule the departure
        global MEAN_SERVICE_1
        TIME_NEXT_EVENT[1] = SIM_TIME + expon(MEAN_SERVICE_1)


def arrive2():
    """Handle the arrival of a customer to Server 2.
    If the server is busy, the customer enters Queue 2.
    If the server is idle, the customer immediately enters Service and
        schedules its Departure.

    If Q1 is empty, S1 must be made idle.
    Otherwise, if Q1 is nonempty, the next customer begins service.
    """

    # handle the customer departing from S1
    global NUM_IN_Q_1, SERVER_STATUS
    if NUM_IN_Q_1 == 0:
        # Q1 is empty, so S1 becomes idle and A2 is impossible
        global TIME_NEXT_EVENT
        SERVER_STATUS[0] = ServiceStatus.IDLE
        TIME_NEXT_EVENT[1] = np.inf

    else:
        # There are customers in Q1, so they start service
        NUM_IN_Q_1 -= 1

        # compute delay of customer starting service S1
        global SIM_TIME, TIME_ARRIVAL_1, TOTAL_OF_DELAYS_1
        delay = SIM_TIME - TIME_ARRIVAL_1[0] # first-in-line
        TOTAL_OF_DELAYS_1 += delay

        # increment the number of customers delayed, schedule next departure
        global NUM_CUSTS_DELAYED_1, MEAN_SERVICE_1
        NUM_CUSTS_DELAYED_1 += 1
        TIME_NEXT_EVENT[1] = SIM_TIME + expon(MEAN_SERVICE_1)

        # move customers up in Q1
        TIME_ARRIVAL_1 = TIME_ARRIVAL_1[1:]

    
    # Handle customer arriving at S2
    delay = 0

    
    if SERVER_STATUS[1] is ServiceStatus.BUSY:
        # if S2 is busy, go to Queue 2
        global NUM_IN_Q_2
        NUM_IN_Q_2 += 1

        # store arrival time of this customer at end of queue
        global TIME_ARRIVAL_2
        TIME_ARRIVAL_2 += [SIM_TIME]

    else:
        # if S2 is idle, go directly to service 2
        delay = 0.0
        global TOTAL_OF_DELAYS_2, NUM_CUSTS_DELAYED_2
        TOTAL_OF_DELAYS_2 += delay
        NUM_CUSTS_DELAYED_2 += 1

        # make server 2 busy
        SERVER_STATUS[1] = ServiceStatus.BUSY

        # schedule departure
        global MEAN_SERVICE_2
        TIME_NEXT_EVENT[2] = SIM_TIME + expon(MEAN_SERVICE_2)


def depart():
    """Handle the departure of a customer from service 2.
    If Q2 is nonempty, the next customer's service is started and
        their departure is scheduled.
    """
    global NUM_IN_Q_2
    if NUM_IN_Q_2 == 0:
        # Q2 is empty
        global SERVER_STATUS, TIME_NEXT_EVENT
        SERVER_STATUS[1] = ServiceStatus.IDLE
        TIME_NEXT_EVENT[2] = np.inf

    else:
        # Q2 is nonempty
        NUM_IN_Q_2 -= 1

        # compute delay
        global SIM_TIME, TIME_ARRIVAL_2, TOTAL_OF_DELAYS_2
        delay = SIM_TIME - TIME_ARRIVAL_2[0]
        TOTAL_OF_DELAYS_2 += delay

        # increment number of customers delayed, schedule the departure
        global NUM_CUSTS_DELAYED_2, MEAN_SERVICE_2
        NUM_CUSTS_DELAYED_2 += 1
        TIME_NEXT_EVENT[2] = SIM_TIME + expon(MEAN_SERVICE_2)

        # move customers up in queue
        TIME_ARRIVAL_2 = TIME_ARRIVAL_2[1:]

def report():
    """Log statistics"""
    global TOTAL_OF_DELAYS_1, TOTAL_OF_DELAYS_2, NUM_CUSTS_DELAYED_1, NUM_CUSTS_DELAYED_2, AREA_NUM_IN_Q_1, AREA_NUM_IN_Q_2, SIM_TIME, AREA_SERVER_STATUS_1, AREA_SERVER_STATUS_2
    print(f"\n\nAverage delay in queue 1 {TOTAL_OF_DELAYS_1 / NUM_CUSTS_DELAYED_1:11.3f} minutes")
    print(f"Average number in queue 1 {AREA_NUM_IN_Q_1 / SIM_TIME:10.3f}")
    print(f"Server 1 utilization {AREA_SERVER_STATUS_1 / SIM_TIME:15.3f}")
    # print(f"Time simulation ended {SIM_TIME:15.3f} minutes")
    print(f"\nAverage delay in queue 2 {TOTAL_OF_DELAYS_2 / NUM_CUSTS_DELAYED_2:11.3f} minutes")
    print(f"Average number in queue 2 {AREA_NUM_IN_Q_2 / SIM_TIME:10.3f}")
    print(f"Server 2 utilization {AREA_SERVER_STATUS_2 / SIM_TIME:15.3f}")
    print(f"\nTime simulation ended {SIM_TIME:15.3f} minutes")


def update_time_avg_stats():
    """Prior to processing each event, this function updates the areas tracked
    for the calculation of continuous-time statistics."""
    global SIM_TIME, TIME_LAST_EVENT
    time_since_last_event = SIM_TIME - TIME_LAST_EVENT
    TIME_LAST_EVENT = SIM_TIME # current time

    # update area under num-in-queue function
    global AREA_NUM_IN_Q_1, NUM_IN_Q_1
    AREA_NUM_IN_Q_1 += NUM_IN_Q_1 * time_since_last_event
    global AREA_NUM_IN_Q_2, NUM_IN_Q_2
    AREA_NUM_IN_Q_2 += NUM_IN_Q_2 * time_since_last_event

    # update area under server-busy indicator function
    global AREA_SERVER_STATUS_1, AREA_SERVER_STATUS_2, SERVER_STATUS
    AREA_SERVER_STATUS_1 += SERVER_STATUS[0].value * time_since_last_event
    AREA_SERVER_STATUS_2 += SERVER_STATUS[1].value * time_since_last_event


def main():
    initialize()

    while True:
        timing()
        update_time_avg_stats()
        
        match NEXT_EVENT_TYPE:
            case 0:
                arrive1()
            case 1:
                arrive2()
            case 2:
                depart()
            case 3:
                report()
                break

main()



Average delay in queue 1       1.639 minutes
Average number in queue 1      1.641
Server 1 utilization           0.731

Average delay in queue 2       7.825 minutes
Average number in queue 2      7.910
Server 2 utilization           0.893

Time simulation ended        1000.000 minutes
